# Migration - Ventilation

In [1]:
import uuid
from main import fake, XMLComplexeElement

xml = fake()
registre = {}
data = {}
data["installations"] = []
data["generateurs"] = []

In [2]:
def get_type_ventilation(enum_type_ventilation_id: int) -> str:
    match enum_type_ventilation_id:
        case 1:
            return "ventilation_naturelle_ouverture_fenetres"
        case 2:
            return "ventilation_naturelle_entrees_air_hautes_basses"
        case 25:
            return "ventilation_naturelle_conduit"
        case 34:
            return "ventilation_naturelle_conduit_entrees_air_hygroreglables"
        case _:
            return "ventilation_mecanique"

In [3]:
def get_type_generateur(enum_type_ventilation_id: int) -> str | None:
    match enum_type_ventilation_id:
        case 3 | 4 | 5 | 6 | 7 | 8 | 9 | 13 | 14 | 15:
            return 'vmc_simple_flux'
        case 10 | 11 | 12:
            return 'vmc_simple_flux_gaz'
        case 16 | 17 | 18:
            return 'vmc_basse_pression'
        case 19 | 20 | 21 | 22 | 23 | 24:
            return 'vmc_double_flux'
        case 26 | 27 | 28 | 29 | 30 | 31:
            return 'ventilation_hybride'
        case 32 | 33:
            return 'ventilation_mecanique'
        case 35 | 36 | 37 | 38:
            return 'puit_climatique'
        case _:
            return None

In [4]:
def get_type_vmc(enum_type_ventilation_id: int, pvent_moy: float | None) -> str | None:
    match enum_type_ventilation_id:
        case 1 | 2 | 25 | 34:
            return None
        case 3 | 4 | 5 | 6 | 16 | 26 | 27 | 28:
            return "autoreglable"
        case 7 | 8 | 9 | 17:
            return "hygroreglable_type_a"
        case 13 | 14 | 15 | 18 | 29 | 30 | 31:
            return "hygroreglable_type_b"
        case _:
            match (pvent_moy or 0):
                case 35 | 65:
                    return "autoreglable"
                case 15 | 50:
                    return "hygroreglable_type_a"
                case 80 | 35:
                    return "hygroreglable_type_b"
                case _:
                    return "autoreglable"

In [5]:
def get_annee_installation_generateur(
    enum_type_ventilation_id: int, annee_etablissement: int
) -> int | None:
    match enum_type_ventilation_id:
        case 3:
            return 1981
        case 4 | 7 | 10 | 13 | 26 | 29:
            return 2000
        case 5 | 8 | 11 | 14 | 19 | 21 | 23 | 27 | 30 | 32 | 35 | 37:
            return 2012
        case 6 | 9 | 12 | 15 | 20 | 22 | 24 | 28 | 31 | 33 | 36 | 38:
            return annee_etablissement
        case _:
            return None

In [6]:
def get_presence_echangeur_thermique(enum_type_ventilation_id: int) -> bool:
    match enum_type_ventilation_id:
        case 19 | 20 | 21 | 22 | 37 | 38:
            return True
        case _:
            return False

In [7]:
def get_generateur_collectif(enum_type_ventilation_id: int) -> bool:
    match enum_type_ventilation_id:
        case 21 | 22:
            return True
        case _:
            return False

In [8]:
for ventilation in xml.logement().ventilation_collection():
    generateur = None
    installation = {}
    installation["id"] = str(uuid.uuid4())
    installation["generateur_id"] = None
    installation["description"] = ventilation.donnee_entree().str("description") or "-"
    installation["surface"] = ventilation.donnee_entree().float("surface_ventile")
    installation["type"] = get_type_ventilation(
        ventilation.donnee_entree().int("enum_type_ventilation_id")
    )

    if installation["type"] == "ventilation_mecanique":
        installation["generateur_id"] = installation["id"]
        generateur = {}
        generateur["id"] = installation["id"]
        generateur["description"] = (
            ventilation.donnee_entree().str("description") or "-"
        )
        generateur["type"] = get_type_generateur(
            ventilation.donnee_entree().int("enum_type_ventilation_id")
        )
        generateur["type_vmc"] = get_type_vmc(
            ventilation.donnee_entree().int("enum_type_ventilation_id"),
            ventilation.donnee_intermediaire().float("pvent_moy"),
        )
        generateur["generateur_collectif"] = get_generateur_collectif(
            ventilation.donnee_entree().int("enum_type_ventilation_id"),
        )
        generateur["presence_echangeur_thermique"] = get_presence_echangeur_thermique(
            ventilation.donnee_entree().int("enum_type_ventilation_id"),
        )
        generateur["annee_installation"] = get_annee_installation_generateur(
            ventilation.donnee_entree().int("enum_type_ventilation_id"),
            xml.administratif().annee_etablissement(),
        )

    data["installations"].append(installation)
    if generateur is not None:
        data["generateurs"].append(generateur)

In [9]:
import json
print(json.dumps(data, indent=2, ensure_ascii=False))

{
  "installations": [
    {
      "id": "78ac7e62-71d5-4cdb-b97f-c15f94b8404b",
      "generateur_id": null,
      "description": "-",
      "surface": 82.0,
      "type": "ventilation_naturelle_ouverture_fenetres"
    },
    {
      "id": "1b9bdc8b-c683-47a2-b3f5-8211d398f0fb",
      "generateur_id": "1b9bdc8b-c683-47a2-b3f5-8211d398f0fb",
      "description": "Ventilation 1",
      "surface": 89.0,
      "type": "ventilation_mecanique"
    }
  ],
  "generateurs": [
    {
      "id": "1b9bdc8b-c683-47a2-b3f5-8211d398f0fb",
      "description": "Ventilation 1",
      "type": "vmc_simple_flux",
      "type_vmc": "autoreglable",
      "generateur_collectif": false,
      "presence_echangeur_thermique": false,
      "annee_installation": 1981
    }
  ]
}
